# Rock Shi

In [81]:
try:
    sc.stop()
except:
    pass

In [82]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('Assignment38').setMaster('local[4]')
sc = SparkContext.getOrCreate(conf = conf)

In [83]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Assignment38').getOrCreate()

### Predicting House Sales

In [84]:
house_df = spark.read.csv('file:///home/hadoop/Downloads/house_sales_data.csv', sep = ',', header = True, inferSchema = 'true')

In [85]:
house_df.show(5)

+----------+---------------+------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|        id|           date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
+----------+---------------+------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|7129300520|20141013T000000|221900|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|
|6414100192|20141209T000000|538000|       3|     2.25|       2570|    7242|   2.0|         0|   0|        3|    7|      2170|          400|    1951|    

In [86]:
house_df.cache()

DataFrame[id: bigint, date: string, price: decimal(7,0), bedrooms: int, bathrooms: double, sqft_living: int, sqft_lot: int, floors: double, waterfront: int, view: int, condition: int, grade: int, sqft_above: int, sqft_basement: int, yr_built: int, yr_renovated: int, zipcode: int, lat: double, long: double, sqft_living15: int, sqft_lot15: int]

In [87]:
from pyspark.mllib.stat import Statistics
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [88]:
house_df = house_df.drop('id','date')
print(house_df.columns)

['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']


In [89]:
def compute_corr_matrix(df, method = 'pearson'):
    df_rdd = df.rdd.map(lambda row: row[0:])
    corr_mat = Statistics.corr(df_rdd, method = method)
    
    corr_mat_df = pd.DataFrame(corr_mat, columns = df.columns, index = df.columns)
    
    return corr_mat_df

compute_corr_matrix(house_df)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
price,1.000000,0.308350,0.525138,0.702035,0.089661,0.256794,0.266369,0.397293,0.036362,0.667434,0.605567,0.323816,0.054012,0.126434,-0.053203,0.307003,0.021626,0.585379,0.082447
bedrooms,0.308350,1.000000,0.515884,0.576671,0.031703,0.175429,-0.006582,0.079532,0.028472,0.356967,0.477600,0.303093,0.154178,0.018841,-0.152668,-0.008931,0.129473,0.391638,0.029244
bathrooms,0.525138,0.515884,1.000000,0.754665,0.087740,0.500653,0.063744,0.187737,-0.124982,0.664983,0.685342,0.283770,0.506019,0.050739,-0.203866,0.024573,0.223042,0.568634,0.087175
sqft_living,0.702035,0.576671,0.754665,1.000000,0.172826,0.353949,0.103818,0.284611,-0.058753,0.762704,0.876597,0.435043,0.318049,0.055363,-0.199430,0.052529,0.240223,0.756420,0.183286
sqft_lot,0.089661,0.031703,0.087740,0.172826,1.000000,-0.005201,0.021604,0.074710,-0.008958,0.113621,0.183512,0.015286,0.053080,0.007644,-0.129574,-0.085683,0.229521,0.144608,0.718557
floors,0.256794,0.175429,0.500653,0.353949,-0.005201,1.000000,0.023698,0.029444,-0.263768,0.458183,0.523885,-0.245705,0.489319,0.006338,-0.059121,0.049614,0.125419,0.279885,-0.011269
waterfront,0.266369,-0.006582,0.063744,0.103818,0.021604,0.023698,1.000000,0.401857,0.016653,0.082775,0.072075,0.080588,-0.026161,0.092885,0.030285,-0.014274,-0.041910,0.086463,0.030703
view,0.397293,0.079532,0.187737,0.284611,0.074710,0.029444,0.401857,1.000000,0.045990,0.251321,0.167649,0.276947,-0.053440,0.103917,0.084827,0.006157,-0.078400,0.280439,0.072575
condition,0.036362,0.028472,-0.124982,-0.058753,-0.008958,-0.263768,0.016653,0.045990,1.000000,-0.144674,-0.158214,0.174105,-0.361417,-0.060618,0.003026,-0.014941,-0.106500,-0.092824,-0.003406
grade,0.667434,0.356967,0.664983,0.762704,0.113621,0.458183,0.082775,0.251321,-0.144674,1.000000,0.755923,0.168392,0.446963,0.014414,-0.184862,0.114084,0.198372,0.713202,0.119248


In [90]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15'],
               outputCol = 'features')
vector_df = vectorAssembler.transform(house_df)
vector_df.show(5)

+------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+--------------------+
| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|            features|
+------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+--------------------+
|221900|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|[3.0,1.0,1180.0,5...|
|538000|       3|     2.25|       2570|    7242|   2.0|         0|   0|        3|    7|      2170|          400|    1951|        1991|  98125| 47.721|-122.319|         1690|   

In [91]:
vector_df_final = vector_df.select(['features','price'])
vector_df_final.show(5)

+--------------------+------+
|            features| price|
+--------------------+------+
|[3.0,1.0,1180.0,5...|221900|
|[3.0,2.25,2570.0,...|538000|
|[2.0,1.0,770.0,10...|180000|
|[4.0,3.0,1960.0,5...|604000|
|[3.0,2.0,1680.0,8...|510000|
+--------------------+------+
only showing top 5 rows



In [92]:
splits = vector_df_final.randomSplit([0.8, 0.2])
train_sample = splits[0]
test_sample = splits[1]

In [93]:
from pyspark.ml.regression import LinearRegression

lr_reg = LinearRegression(featuresCol = 'features', labelCol = 'price', maxIter = 100, 
                         regParam = 0.3, elasticNetParam = 0.7)

lr_model = lr_reg.fit(train_sample)

In [94]:
trainingSummary = lr_model.summary
print("RMSE: %f " %trainingSummary.rootMeanSquaredError)
print("r2 %f " %trainingSummary.r2)

RMSE: 203960.285190 
r2 0.703265 


In [95]:
lr_model.summary.meanSquaredError

41599797934.839485

In [96]:
lr_model.summary.meanAbsoluteError

127615.77092901336

In [97]:
lr_predictions = lr_model.transform(test_sample)
lr_predictions.select('prediction', 'price', 'features').show(10)

+-------------------+------+--------------------+
|         prediction| price|            features|
+-------------------+------+--------------------+
| -309773.6467118114|280000|[1.0,0.0,600.0,24...|
| -426717.9515646994| 75000|[1.0,0.0,670.0,43...|
|  229092.0051649781|276000|[1.0,0.75,370.0,1...|
|-244630.58031739946|262000|[1.0,0.75,520.0,1...|
| -72383.53455910087|299000|[1.0,0.75,560.0,1...|
| 166531.37673244718|200000|[1.0,0.75,680.0,9...|
| 125040.86217730306|224000|[1.0,0.75,840.0,7...|
| 446430.79495441075|170000|[1.0,0.75,850.0,5...|
| 373700.73594505806|351000|[1.0,0.75,930.0,6...|
| 151752.25309184194|190000|[1.0,0.75,930.0,2...|
+-------------------+------+--------------------+
only showing top 10 rows



In [98]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol = "prediction", \
                                  labelCol = 'price', metricName = "r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

R Squared (R2) on test data = 0.6796


In [99]:
test_result = lr_model.evaluate(test_sample)
print("Root Mean Squared Error(RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error(RMSE) on test data = 190086


### Predicting Medical Insurance Cost

In [100]:
insurance_df = spark.read.csv('file:///home/hadoop/Downloads/insurance.csv', sep = ',', header = True, inferSchema = 'true')

In [101]:
insurance_df.cache()

DataFrame[age: int, sex: string, bmi: double, children: int, smoker: string, region: string, charges: double]

In [102]:
insurance_pd_df = insurance_df.toPandas()
gender = pd.get_dummies(insurance_pd_df['sex'])
insurance_pd_df = pd.concat([insurance_pd_df, gender], axis = 1)
insurance_pd_df.drop(['sex'], axis = 1, inplace = True)

smoker = pd.get_dummies(insurance_pd_df['smoker'])
insurance_pd_df = pd.concat([insurance_pd_df, smoker], axis = 1)
insurance_pd_df.drop(['smoker'], axis = 1, inplace = True)

insurance_pd_df.drop(['region', 'female', 'no'], axis = 1, inplace = True)
insurance_pd_df.rename(columns = {"yes":'smoker'}, inplace = True)

insurance_df = spark.createDataFrame(insurance_pd_df)


In [103]:
print(insurance_df.columns)

['age', 'bmi', 'children', 'charges', 'male', 'smoker']


In [104]:
compute_corr_matrix(insurance_df)

,age,bmi,children,charges,male,smoker
age,1.000000,0.109272,0.042469,0.299008,-0.020856,-0.025019
bmi,0.109272,1.000000,0.012759,0.198341,0.046371,0.003750
children,0.042469,0.012759,1.000000,0.067998,0.017163,0.007673
charges,0.299008,0.198341,0.067998,1.000000,0.057292,0.787251
male,-0.020856,0.046371,0.017163,0.057292,1.000000,0.076185
smoker,-0.025019,0.003750,0.007673,0.787251,0.076185,1.000000


In [105]:
vectorAssembler = VectorAssembler(inputCols = ['age', 'bmi', 'children', 'male', 'smoker'],
               outputCol = 'features')
vector_df = vectorAssembler.transform(insurance_df)
vector_df.show(5)

+---+------+--------+-----------+----+------+--------------------+
|age|   bmi|children|    charges|male|smoker|            features|
+---+------+--------+-----------+----+------+--------------------+
| 19|  27.9|       0|  16884.924|   0|     1|[19.0,27.9,0.0,0....|
| 18| 33.77|       1|  1725.5523|   1|     0|[18.0,33.77,1.0,1...|
| 28|  33.0|       3|   4449.462|   1|     0|[28.0,33.0,3.0,1....|
| 33|22.705|       0|21984.47061|   1|     0|[33.0,22.705,0.0,...|
| 32| 28.88|       0|  3866.8552|   1|     0|[32.0,28.88,0.0,1...|
+---+------+--------+-----------+----+------+--------------------+
only showing top 5 rows



In [106]:
vector_df_final = vector_df.select(['features','charges'])
vector_df_final.show(5)

+--------------------+-----------+
|            features|    charges|
+--------------------+-----------+
|[19.0,27.9,0.0,0....|  16884.924|
|[18.0,33.77,1.0,1...|  1725.5523|
|[28.0,33.0,3.0,1....|   4449.462|
|[33.0,22.705,0.0,...|21984.47061|
|[32.0,28.88,0.0,1...|  3866.8552|
+--------------------+-----------+
only showing top 5 rows



In [107]:
splits = vector_df_final.randomSplit([0.8, 0.2])
train_sample = splits[0]
test_sample = splits[1]

In [108]:
lr_reg = LinearRegression(featuresCol = 'features', labelCol = 'charges', maxIter = 100, 
                         regParam = 0.3, elasticNetParam = 0.7)

lr_model = lr_reg.fit(train_sample)

In [109]:
trainingSummary = lr_model.summary
print("RMSE: %f " %trainingSummary.rootMeanSquaredError)
print("r2 %f " %trainingSummary.r2)

RMSE: 5974.517296 
r2 0.744881 


In [110]:
lr_model.summary.meanSquaredError

35694856.92221533

In [111]:
lr_model.summary.meanAbsoluteError

4086.1749878879036

In [112]:
lr_predictions = lr_model.transform(test_sample)
lr_predictions.select('prediction', 'charges', 'features').show(10)

+-------------------+----------+--------------------+
|         prediction|   charges|            features|
+-------------------+----------+--------------------+
|  2168.443052700619|  1743.214|(5,[0,1],[19.0,28...|
| 2393.7644111567733|2457.21115|(5,[0,1],[20.0,28...|
|    5715.5624908803| 2026.9741|(5,[0,1],[21.0,39...|
|  2699.885802353736| 2155.6815|(5,[0,1],[22.0,28...|
|  6063.941611826744|2755.02095|(5,[0,1],[22.0,39...|
|  5785.701418419725|  4883.866|(5,[0,1],[36.0,26...|
|  9677.576661601459| 5920.1041|(5,[0,1],[40.0,36...|
| 15377.377946037377|12648.7034|(5,[0,1],[60.0,38...|
|-1837.0686602724982| 1694.7964|[18.0,15.96,0.0,1...|
| 23564.256876902247|12829.4551|[18.0,17.29,2.0,1...|
+-------------------+----------+--------------------+
only showing top 10 rows



In [113]:
lr_evaluator = RegressionEvaluator(predictionCol = "prediction", \
                                  labelCol = 'charges', metricName = "r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

R Squared (R2) on test data = 0.761905


In [114]:
test_result = lr_model.evaluate(test_sample)
print("Root Mean Squared Error(RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error(RMSE) on test data = 6399.05


### Predicting Automobile Average

In [175]:
from pyspark.sql.types import *
schema = StructType([
    StructField('mpg', FloatType()),
    StructField('cylinder', IntegerType()),
    StructField('displacements', FloatType()),
    StructField('horsepower', FloatType()),
    StructField('weight', FloatType()),
    StructField('acceleration', FloatType()),
    StructField('model_year', IntegerType()),
    StructField('origin', IntegerType()),
    StructField('car_name', StringType()),
    
])

In [176]:
auto_df = spark.read.csv('file:///home/hadoop/Downloads/auto-mpg.csv', schema = schema)

In [177]:
auto_df.cache()

DataFrame[mpg: float, cylinder: int, displacements: float, horsepower: float, weight: float, acceleration: float, model_year: int, origin: int, car_name: string]

In [178]:
auto_df = auto_df.drop('car_name')
print(auto_df.columns)

['mpg', 'cylinder', 'displacements', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin']


In [179]:
vectorAssembler = VectorAssembler(inputCols = ['cylinder', 'displacements', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin'],
               outputCol = 'features')
vector_df = vectorAssembler.transform(auto_df)
vector_df.show(5)

+----+--------+-------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinder|displacements|horsepower|weight|acceleration|model_year|origin|            features|
+----+--------+-------------+----------+------+------------+----------+------+--------------------+
|18.0|       8|        307.0|     130.0|3504.0|        12.0|        70|     1|[8.0,307.0,130.0,...|
|15.0|       8|        350.0|     165.0|3693.0|        11.5|        70|     1|[8.0,350.0,165.0,...|
|18.0|       8|        318.0|     150.0|3436.0|        11.0|        70|     1|[8.0,318.0,150.0,...|
|16.0|       8|        304.0|     150.0|3433.0|        12.0|        70|     1|[8.0,304.0,150.0,...|
|17.0|       8|        302.0|     140.0|3449.0|        10.5|        70|     1|[8.0,302.0,140.0,...|
+----+--------+-------------+----------+------+------------+----------+------+--------------------+
only showing top 5 rows



In [180]:
vector_df_final = vector_df.select(['features','mpg'])
vector_df_final.show(5)
vector_df_final.count()

+--------------------+----+
|            features| mpg|
+--------------------+----+
|[8.0,307.0,130.0,...|18.0|
|[8.0,350.0,165.0,...|15.0|
|[8.0,318.0,150.0,...|18.0|
|[8.0,304.0,150.0,...|16.0|
|[8.0,302.0,140.0,...|17.0|
+--------------------+----+
only showing top 5 rows



392

In [181]:
splits = vector_df_final.randomSplit([0.8, 0.2])
train_sample = splits[0]
test_sample = splits[1]

306


In [182]:
lr_reg = LinearRegression(featuresCol = 'features', labelCol = 'mpg', maxIter = 100, 
                         regParam = 0.3, elasticNetParam = 0.7)

lr_model = lr_reg.fit(train_sample)

In [183]:
trainingSummary = lr_model.summary
print("RMSE: %f " %trainingSummary.rootMeanSquaredError)
print("r2 %f " %trainingSummary.r2)

RMSE: 3.297225 
r2 0.817756 


In [184]:
lr_model.summary.meanSquaredError

10.871692366013173

In [185]:
lr_model.summary.meanAbsoluteError

2.472136387488705